In [ ]:
import datetime as dt
import numpy as np
import numba as nb
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gas_storage.gas_price_simulations import GasPriceSimulations

# NOW!
Run the MILP optimizer on the forward curve, understand and recall everything, then go back to this crazy script with LSMC method and compare what it actually gives (the control coming from stochastic method and control from MILP).

In [ ]:
simulator = GasPriceSimulations()
paths = simulator.get_simulations(number_of_simulations=1000)
fig = go.Figure()
for path in paths[:50, :]:
    fig.add_trace(
        go.Scatter(
            x=simulator.index, y=path, showlegend=False, line=dict(color="lightgray")
        )
    )
fig = fig.update_layout(template="plotly_dark", title="Gas spot price paths")
# fig.show()

### LSMC + Bellman Backward Induction for Gas Storage

At each time step $t$ (backward from $T-1$ to $0$), for each inventory level $I$:

1. **Regression** — For each reachable next-state $I' = I + a$, estimate:
$$\hat{\mathbb{E}}[V_{t+1}(I') \mid S_t] \approx \hat{\beta}_0 + \hat{\beta}_1 S_t + \hat{\beta}_2 S_t^2$$
by regressing the pathwise $V_{t+1}^{(n)}(I')$ on the basis $[1,\; S_t,\; S_t^2]$.

2. **Action selection** — Choose optimal action using **fitted** continuation values (avoids look-ahead):
$$a^* = \arg\max_{a \in \mathcal{A}(I)} \left\{-a \cdot S_t + \hat{\mathbb{E}}[V_{t+1}(I+a) \mid S_t]\right\}$$

3. **Realized value** — Record the **actual** (pathwise) value under the chosen action:
$$V_t^{(n)}(I) = -a^{*(n)} \cdot S_t^{(n)} + V_{t+1}^{(n)}(I + a^{*(n)})$$

This feeds into the regression at the next backward step. Using fitted values to **decide** but actual values to **record** prevents compounding of regression errors.

In [ ]:
import time

# === Storage Parameters ===
I_max = 100          # Maximum inventory (MWh)
delta_I = 10         # Inventory grid step (MWh)
q_in = delta_I       # Max injection per day (one grid step)
q_out = delta_I      # Max withdrawal per day (one grid step)

inventory_levels = np.arange(0, I_max + delta_I, delta_I)
n_levels = len(inventory_levels)

# Time horizon: first year of simulated paths
T = 365
N = paths.shape[0]
S = paths[:, :T + 1]              # shape (N, T+1)
dates = simulator.index[:T + 1]

# Precompute feasible actions: (action_value, next_inventory_index)
action_map = []
for i_idx, I_val in enumerate(inventory_levels):
    actions = []
    if I_val >= q_out:
        actions.append((-q_out, i_idx - 1))   # withdraw
    actions.append((0, i_idx))                  # hold
    if I_val + q_in <= I_max:
        actions.append((q_in, i_idx + 1))       # inject
    action_map.append(actions)

print(f"Inventory levels ({n_levels}): {inventory_levels.tolist()}")
print(f"Horizon: {T} days  ({dates[0].strftime('%Y-%m-%d')} → {dates[-1].strftime('%Y-%m-%d')})")
print(f"Simulations: {N}")
for i_idx in [0, n_levels // 2, n_levels - 1]:
    I = inventory_levels[i_idx]
    acts = [(a, inventory_levels[j]) for a, j in action_map[i_idx]]
    print(f"  A({I:3d}) = {acts}")

In [ ]:
# === LSMC Backward Induction ===

# Terminal condition: V_T(I) = I * S_T  (liquidate remaining gas at spot)
V_next = np.zeros((N, n_levels))
for i_idx, I_val in enumerate(inventory_levels):
    V_next[:, i_idx] = I_val * S[:, T]

# Store regression coefficients for forward pass: (T, n_levels, n_basis)
reg_coeffs = np.zeros((T, n_levels, 3))

t0 = time.time()

for t in range(T - 1, -1, -1):
    St = S[:, t]
    X = np.column_stack([np.ones(N), St, St**2])

    # Vectorised regression: fit E[V_{t+1}(I') | S_t] for ALL inventory levels at once
    pinv_X = np.linalg.pinv(X)          # (3, N)
    betas = pinv_X @ V_next             # (3, n_levels)
    EV_next = X @ betas                  # (N, n_levels)  — fitted continuation values
    reg_coeffs[t] = betas.T             # store for forward pass

    # For each inventory level: pick best action, record realised value
    V_current = np.zeros((N, n_levels))

    for i_idx in range(n_levels):
        actions = action_map[i_idx]

        # Q-values using FITTED continuation (prevents look-ahead bias)
        Q_vals = np.column_stack([-a * St + EV_next[:, j] for a, j in actions])
        best_k = np.argmax(Q_vals, axis=1)

        # REALISED value under optimal action (actual V_next, NOT fitted)
        for k, (a, j) in enumerate(actions):
            mask = best_k == k
            if mask.any():
                V_current[mask, i_idx] = -a * St[mask] + V_next[mask, j]

    V_next = V_current

elapsed = time.time() - t0
print(f"Backward induction completed in {elapsed:.1f}s\n")

print("Backward-pass storage values (upper bound):")
for i_idx, I_val in enumerate(inventory_levels):
    print(f"  V_0({I_val:3d}) = {np.mean(V_next[:, i_idx]):>10,.2f}")

In [ ]:
# === Forward Simulation (unbiased / lower-bound estimate) ===

I_start = 0
i_start = int(I_start / delta_I)

inv_idx = np.full(N, i_start, dtype=int)
cash_flows = np.zeros((N, T + 1))
actions_log = np.zeros((N, T))
inv_log = np.zeros((N, T + 1))
inv_log[:, 0] = I_start

for t in range(T):
    St = S[:, t]
    X = np.column_stack([np.ones(N), St, St**2])

    action = np.zeros(N)
    next_inv = inv_idx.copy()

    for i_idx in range(n_levels):
        mask = inv_idx == i_idx
        if not mask.any():
            continue

        actions = action_map[i_idx]
        X_m, St_m = X[mask], St[mask]

        # Q-values using stored regression coefficients (no future info)
        Q_vals = np.column_stack([-a * St_m + X_m @ reg_coeffs[t, j] for a, j in actions])
        best_k = np.argmax(Q_vals, axis=1)

        for k, (a, j) in enumerate(actions):
            sel = best_k == k
            if sel.any():
                full_sel = mask.copy()
                full_sel[mask] = sel
                action[full_sel] = a
                next_inv[full_sel] = j

    inv_idx = next_inv
    cash_flows[:, t] = -action * St
    actions_log[:, t] = action
    inv_log[:, t + 1] = inventory_levels[inv_idx]

# Terminal liquidation
cash_flows[:, T] = inv_log[:, T] * S[:, T]

path_values = cash_flows.sum(axis=1)
fwd_value = np.mean(path_values)
fwd_se = np.std(path_values) / np.sqrt(N)

print(f"Forward simulation  (I₀ = {I_start}):")
print(f"  Storage value : {fwd_value:>10,.2f}")
print(f"  Std error     : {fwd_se:>10,.2f}")
print(f"  95% CI        : [{fwd_value - 1.96*fwd_se:,.2f},  {fwd_value + 1.96*fwd_se:,.2f}]")
print(f"  Avg terminal I: {inv_log[:, T].mean():.1f}")

In [ ]:
# === Visualization ===

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Average inventory & spot price",
        "Cumulative P&L (mean ± P10/P90)",
        "Storage value distribution",
        "Sample inventory paths",
    ],
    specs=[[{"secondary_y": True}, {}], [{}, {}]],
)

avg_inv = inv_log.mean(axis=0)
avg_price = S.mean(axis=0)

# 1 — Inventory + price (dual y-axis)
fig.add_trace(
    go.Scatter(x=dates, y=avg_inv, name="Avg inventory", line=dict(color="cyan", width=2)),
    row=1, col=1, secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=dates, y=avg_price, name="Avg price", line=dict(color="orange", width=2)),
    row=1, col=1, secondary_y=True,
)
fig.update_yaxes(title_text="Inventory (MWh)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Price (EUR/MWh)", row=1, col=1, secondary_y=True)

# 2 — Cumulative P&L
cum_cf = np.cumsum(cash_flows[:, :T], axis=1)
avg_cf = cum_cf.mean(axis=0)
q10, q90 = np.percentile(cum_cf, [10, 90], axis=0)

fig.add_trace(go.Scatter(x=dates[:T], y=avg_cf, name="Mean P&L", line=dict(color="lime", width=2)), row=1, col=2)
fig.add_trace(go.Scatter(x=dates[:T], y=q90, line=dict(dash="dash", color="gray"), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=dates[:T], y=q10, line=dict(dash="dash", color="gray"), showlegend=False, fill="tonexty"), row=1, col=2)

# 3 — Value distribution
fig.add_trace(go.Histogram(x=path_values, nbinsx=50, name="Path values", marker_color="orange"), row=2, col=1)

# 4 — Sample inventory paths
for i in range(min(30, N)):
    fig.add_trace(go.Scatter(x=dates, y=inv_log[i], showlegend=False, line=dict(color="lightblue", width=0.5)), row=2, col=2)
fig.add_trace(go.Scatter(x=dates, y=avg_inv, name="Avg", line=dict(color="cyan", width=3)), row=2, col=2)

fig.update_layout(
    template="plotly_dark", height=800, width=1200,
    title=f"Gas Storage LSMC — Value: {fwd_value:,.0f} EUR  (I₀={I_start}, T={T}d, ΔI={delta_I})",
)
fig.show()